In [1]:
import xarray as xr
import numpy as np
from my_funs import est_trend
import pandas as pd

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/working/"

def outliers_index(data, m=3.5):
    """
    Returns true if a value is outlier

    :param int data: numpy array
    :param int m: # of std to include data 
    """
    import numpy as np
    d = np.abs(data - np.nanmedian(data))
    mdev = np.nanmedian(d)
    s = d / mdev if mdev else 0.
    return ~(s < m)

def normalize(xr):
    # Notalize to [0 1] along time dimension
    xr_norm = (
        (xr - xr.min(dim="time")) / (xr.max(dim="time") - xr.min(dim="time"))
    )
    return xr_norm


/data/home/hamiddashti/hamid/nasa_above/greeness/codes/src/my_funs.py:9: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


Estimating trend for maximum LAI

In [2]:
# Prepare LAI MAX data
lai_max = xr.open_dataarray(
    dir + "data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max.nc"
).rename({"latitude":"lat","longitude":"lon"})
# lai_max["time"] = pd.date_range("1984","2014",freq = "A")
lai_max = lai_max.sel(time=slice("1985","2014"))  # 1984 has many nan values
# Prepare NDVI MAX data
ndvi_max = xr.open_dataarray(
    dir + "data/processed_data/landsat/resampled/ndvi_annual_max.nc"
).rename({"latitude":"lat","longitude":"lon"})*1e-4
ndvi_max = ndvi_max.sel(time=slice("1985","2014"))

aut = xr.open_dataarray(dir+"data/processed_data/landsat/resampled/aut.nc")
spr = xr.open_dataarray(dir+"data/processed_data/landsat/resampled/spr.nc")
gs = aut-spr #Growing season

# Remove outliers
I_phen = outliers_index(gs,3.5)
I_lai = outliers_index(lai_max,3.5)
I_ndvi = outliers_index(ndvi_max,3.5)
gs_clean=gs.where(I_phen==False)
lai_clean = lai_max.where(I_lai==False)
ndvi_clean = ndvi_max.where(I_lai==False)

# Normalize data to [0 1]
lai_norm = normalize(lai_clean)
ndvi_norm = normalize(ndvi_clean)
gs_norm = normalize(gs_clean)

lai_norm.to_netcdf(dir+"data/processed_data/noaa_nc/lai_fapar/resampled/lai_annual_resample_max_norm.nc")
ndvi_norm.to_netcdf(dir+"data/processed_data/landsat/resampled/ndvi_annual_max_norm.nc")
gs_norm.to_netcdf(dir+"data/processed_data/landsat/resampled/gs_norm.nc")
gs.to_netcdf(dir+"data/processed_data/landsat/resampled/gs.nc")

In [9]:
est_trend_lai_max = est_trend(lai_clean, "theilsen")
est_trend_ndvi_max = est_trend(ndvi_clean, "theilsen")
est_trend_gs = est_trend(gs_clean,"theilsen")

est_trend_lai_max_norm = est_trend(lai_norm,"theilsen")
est_trend_ndvi_max_norm = est_trend(ndvi_norm,"theilsen")
est_trend_gs_norm = est_trend(gs_norm,"theilsen")

In [24]:
est_trend_lai_max.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_max_trend.nc"
)
est_trend_ndvi_max.to_netcdf(
    dir + "data/processed_data/landsat/trend/ndvi_annual_max_trend.nc")
est_trend_gs.to_netcdf(dir+"data/processed_data/landsat/trend/gs_trend.nc")

est_trend_lai_max_norm.to_netcdf(dir+"data/processed_data/noaa_nc/lai_fapar/trend/lai_max_norm_trend.nc")
est_trend_ndvi_max_norm.to_netcdf(dir+"data/processed_data/landsat/trend/ndvi_max_norm_trend.nc")
est_trend_gs_norm.to_netcdf(dir+"data/processed_data/landsat/trend/gs_trend_norm.nc")